In [1]:
import sys
sys.path.append('../..')
import os
import json
from pathlib import Path
import shutil


import numpy as np
import math
import pandas as pd
import tensorflow as tf
import seaborn as sns
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from tqdm.keras import TqdmCallback

from src.Evaluation import plot_model_history
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder, get_Resnet
from src.LoadData import get_all_datasets_test_train_np_arrays, get_all_datasets_names_paths
from src.Helpers import append_to_csv, get_confusion_matrix_for_model_and_data, visualize_confusion_matrix

2023-01-24 11:44:45.047275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 11:44:45.169502: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 11:44:46.095433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-24 11:44:46.095500: W tensorflow/compil

In [2]:
path_to_ds_analysis_file = "../training_res.csv"
path_to_datasets = "../../datasets"
path_to_persiste = "./ds_confusion_matrices_history"
persist_history = True

In [3]:
training_res_df = pd.read_csv(path_to_ds_analysis_file)

In [4]:
datasets_names, _ = get_all_datasets_names_paths(path_to_datasets)

In [5]:
for dataset_name in datasets_names:
    print("Dataset name", dataset_name)
    df_dataset = training_res_df[training_res_df["dataset_name"] == dataset_name]
    num_conf_matr = len(df_dataset)
    if num_conf_matr > 0:
        
        curr_persist_dir = Path(path_to_persiste) / dataset_name
        #curr_persist_dir = os.path.join(path_to_persiste, dataset_name)
        if curr_persist_dir.exists() and curr_persist_dir.is_dir():
            shutil.rmtree(curr_persist_dir)
        os.mkdir(curr_persist_dir)
        
        num_horizontal = 2 
        num_vertical = math.ceil(num_conf_matr/num_horizontal)

        figure, axis = plt.subplots(num_horizontal, num_vertical)
        figure.suptitle(f"Dataset {dataset_name}", fontsize=16)
        plt.figure(figsize=(15, 15))

        conf_matrices = [json.loads(mtr) for i, mtr in df_dataset["confusion_matrix"].iteritems()]
        test_acc = [round(acc, 2) for i, acc in df_dataset["test_acc"].iteritems()]
        model_names = df_dataset["model_name"].values.tolist()
        curr_matr = 0

        for i in range(num_horizontal):
            for j in range(num_vertical):
                curr_axis = axis[i, j]
                if curr_matr < len(conf_matrices):
                    sns.heatmap(conf_matrices[curr_matr], annot=True, fmt="d", ax=curr_axis)
                    curr_axis.set_title(f"{model_names[curr_matr]}; \n Test accuracy: {test_acc[curr_matr]}")
                    curr_axis.set_ylabel('True label')
                    curr_axis.set_xlabel('Predicted label')
                curr_matr += 1

        figure.tight_layout()            
        figure.savefig(os.path.join(curr_persist_dir, "confusion_matrix.png"))
        plt.close(figure)
        
        if persist_history:
            for _, row in df_dataset.iterrows():
                history = json.loads(row["history"])
                epochs = len(history["loss"])
                model_name = row["model_name"]
                plot_model_history(history=history, epochs=epochs, path_to_persist=os.path.join(curr_persist_dir, f"history_{model_name}.png"))
        

Dataset name cbf


/scratch/slurm_tmpdir/job_21679853/ipykernel_152942/2969229585.py:20: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  conf_matrices = [json.loads(mtr) for i, mtr in df_dataset["confusion_matrix"].iteritems()]
/scratch/slurm_tmpdir/job_21679853/ipykernel_152942/2969229585.py:21: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  test_acc = [round(acc, 2) for i, acc in df_dataset["test_acc"].iteritems()]


Dataset name gun_point_male_female
Dataset name car
Dataset name abnormal_heartbeat
Dataset name distal_phalanax_tw
Dataset name phalanges_outlines_correct
Dataset name swedish_leaf
Dataset name share_price_increase
Dataset name distal_phalanx_outline
Dataset name strawberry
Dataset name earthquakes
Dataset name electric_devices
Dataset name fifty_words
Dataset name freezers
Dataset name synthetic_control
Dataset name gun_point_age_span
Dataset name trace
Dataset name fish
Dataset name middle_phalanx_age_group
Dataset name lightning_7


/scratch/slurm_tmpdir/job_21679853/ipykernel_152942/2969229585.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 15))
../../src/Evaluation.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  figure, axis = plt.subplots(2, 1)


Dataset name colposcopy


...

<Figure size 1500x1500 with 0 Axes>

<Figure size 1500x1500 with 0 Axes>

<Figure size 1500x1500 with 0 Axes>

<Figure size 1500x1500 with 0 Axes>

<Figure size 1500x1500 with 0 Axes>